In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark =  SparkSession \
        .builder \
        .appName('pyspark-init') \
        .enableHiveSupport() \
        .getOrCreate()

- enableHiveSupport() X : 임시 테이블만 생성되고 CREATE TABLE 쿼리는 적용이 안된다.
- enableHiveSupport() O : hive로 영구 저장소가 생성되며 DB 처럼 다룰 수 있다. 세션 종료되어도 테이블 조회 가능

In [6]:
cars = spark.read.option("inferSchema", "true").json("./dataset/cars.json")
cars.show(3)

+------------+---------+------------+----------+----------------+--------------------+------+-------------+----------+
|Acceleration|Cylinders|Displacement|Horsepower|Miles_per_Gallon|                Name|Origin|Weight_in_lbs|      Year|
+------------+---------+------------+----------+----------------+--------------------+------+-------------+----------+
|        12.0|        8|       307.0|       130|            18.0|chevrolet chevell...|   USA|         3504|1970-01-01|
|        11.5|        8|       350.0|       165|            15.0|   buick skylark 320|   USA|         3693|1970-01-01|
|        11.0|        8|       318.0|       150|            18.0|  plymouth satellite|   USA|         3436|1970-01-01|
+------------+---------+------------+----------+----------------+--------------------+------+-------------+----------+
only showing top 3 rows



In [9]:
cars.select("*", F.expr("Cylinders * 2 AS aa")).show(5)

+------------+---------+------------+----------+----------------+--------------------+------+-------------+----------+---+
|Acceleration|Cylinders|Displacement|Horsepower|Miles_per_Gallon|                Name|Origin|Weight_in_lbs|      Year| aa|
+------------+---------+------------+----------+----------------+--------------------+------+-------------+----------+---+
|        12.0|        8|       307.0|       130|            18.0|chevrolet chevell...|   USA|         3504|1970-01-01| 16|
|        11.5|        8|       350.0|       165|            15.0|   buick skylark 320|   USA|         3693|1970-01-01| 16|
|        11.0|        8|       318.0|       150|            18.0|  plymouth satellite|   USA|         3436|1970-01-01| 16|
|        12.0|        8|       304.0|       150|            16.0|       amc rebel sst|   USA|         3433|1970-01-01| 16|
|        10.5|        8|       302.0|       140|            17.0|         ford torino|   USA|         3449|1970-01-01| 16|
+------------+--

In [10]:
cars.selectExpr("*", "Cylinders * 2 AS aa").show(5)

+------------+---------+------------+----------+----------------+--------------------+------+-------------+----------+---+
|Acceleration|Cylinders|Displacement|Horsepower|Miles_per_Gallon|                Name|Origin|Weight_in_lbs|      Year| aa|
+------------+---------+------------+----------+----------------+--------------------+------+-------------+----------+---+
|        12.0|        8|       307.0|       130|            18.0|chevrolet chevell...|   USA|         3504|1970-01-01| 16|
|        11.5|        8|       350.0|       165|            15.0|   buick skylark 320|   USA|         3693|1970-01-01| 16|
|        11.0|        8|       318.0|       150|            18.0|  plymouth satellite|   USA|         3436|1970-01-01| 16|
|        12.0|        8|       304.0|       150|            16.0|       amc rebel sst|   USA|         3433|1970-01-01| 16|
|        10.5|        8|       302.0|       140|            17.0|         ford torino|   USA|         3449|1970-01-01| 16|
+------------+--

In [2]:
spark.conf.get("spark.sql.catalogImplementation")

'hive'

In [2]:
spark.sql("select * from cars").show()

AnalysisException: Table or view not found: cars; line 1 pos 14;
'Project [*]
+- 'UnresolvedRelation [cars], [], false


In [16]:
cars.createOrReplaceTempView("cars")

In [8]:
cars.write.saveAsTable("cars")

In [7]:
spark.sql("show tables from default").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



In [21]:
spark.stop()

In [4]:
# spark.sql("CREATE DATABASE temp")
spark.sql("SHOW DATABASES").show()

+---------+
|namespace|
+---------+
|  default|
|     temp|
+---------+



In [6]:
spark.sql("CREATE TABLE temp.person2 (id Int, name String)")

DataFrame[]

In [8]:
spark.sql("show tables from temp").show()

+---------+----------+-----------+
|namespace| tableName|isTemporary|
+---------+----------+-----------+
|     temp|movies_per|      false|
|     temp|    person|      false|
|     temp|   person2|      false|
+---------+----------+-----------+



In [7]:
spark.sql("INSERT INTO temp.person2 VALUES (1, 'Tom'), (2, 'Ann')")

DataFrame[]

In [30]:
spark.sql("select * from temp.person").show()

+---+----+
| id|name|
+---+----+
|  1| Tom|
|  2| Ann|
+---+----+



In [31]:
cars.show(1)

+------------+---------+------------+----------+----------------+--------------------+------+-------------+----------+
|Acceleration|Cylinders|Displacement|Horsepower|Miles_per_Gallon|                Name|Origin|Weight_in_lbs|      Year|
+------------+---------+------------+----------+----------------+--------------------+------+-------------+----------+
|        12.0|        8|       307.0|       130|            18.0|chevrolet chevell...|   USA|         3504|1970-01-01|
+------------+---------+------------+----------+----------------+--------------------+------+-------------+----------+
only showing top 1 row



In [14]:
cars.createOrReplaceGlobalTempView("cars_global")

cars.createGlobalTempView("cars_g")

In [35]:
cars.createOrReplaceTempView("cars")

In [16]:
spark.sql("show tables from temp").show()

+---------+----------+-----------+
|namespace| tableName|isTemporary|
+---------+----------+-----------+
|     temp|movies_per|      false|
|     temp|    person|      false|
+---------+----------+-----------+



In [4]:
movies = spark.read.option("inferSchema", "true").json("./dataset/movies.json")

- createOrReplaceTempView 이걸로 생성하면 DB 모두에 다 생성된다.
- saveAsTable 및 read.table의 경우 db 명시 안하면 default에 저장, db.table 이렇게 명시해서 저장/읽기 하면 특정 db에 저장됨

In [54]:
movies.createOrReplaceTempView("temp.movies2")

In [45]:
movies.write.saveAsTable("temp.movies_per")

In [51]:
spark.read.table("temp.movies_per").show(5)

+--------------------+--------+-----------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+-------------------+--------------------+------------+--------+---------------+
|       Creative_Type|Director|Distributor|IMDB_Rating|IMDB_Votes|MPAA_Rating|Major_Genre|Production_Budget|Release_Date|Rotten_Tomatoes_Rating|Running_Time_min|             Source|               Title|US_DVD_Sales|US_Gross|Worldwide_Gross|
+--------------------+--------+-----------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+-------------------+--------------------+------------+--------+---------------+
|                null|    null|   Gramercy|        6.1|      1071|          R|       null|          8000000|   12-Jun-98|                  null|            null|               null|      The Land Girls|        null|  146083|         146083|
|                null|    null|     

### repartition

In [9]:
movies = spark.read.option("inferSchema", "true").json("./dataset/movies.json")

In [10]:
mv = movies.repartition(4)

In [11]:
mv.rdd.getNumPartitions()

4

In [12]:
mv.withColumn('part', F.spark_partition_id()).drop_duplicates(subset=['part']).show()

+--------------------+--------------------+--------------------+-----------+----------+-----------+------------+-----------------+------------+----------------------+----------------+--------------------+--------------------+------------+---------+---------------+----+
|       Creative_Type|            Director|         Distributor|IMDB_Rating|IMDB_Votes|MPAA_Rating| Major_Genre|Production_Budget|Release_Date|Rotten_Tomatoes_Rating|Running_Time_min|              Source|               Title|US_DVD_Sales| US_Gross|Worldwide_Gross|part|
+--------------------+--------------------+--------------------+-----------+----------+-----------+------------+-----------------+------------+----------------------+----------------+--------------------+--------------------+------------+---------+---------------+----+
|        Kids Fiction|         Mark Dindal|Walt Disney Pictures|        7.4|     23355|          G|   Adventure|        100000000|   15-Dec-00|                    85|              78| Origin

In [19]:
movies = spark.read.option("inferSchema", "true").json("./dataset/movies.json")

movies2 = movies.repartition(6, "Major_Genre")

In [18]:
movies.groupBy("Major_Genre").count().count()

13

In [14]:
movies2.rdd.getNumPartitions()

6

In [20]:
movies2.withColumn('part', F.spark_partition_id()).drop_duplicates(subset=['Major_Genre','part']).select('Major_Genre', 'part').show()

+-------------------+----+
|        Major_Genre|part|
+-------------------+----+
|               null|   0|
|             Comedy|   0|
|       Black Comedy|   0|
|            Musical|   1|
|  Thriller/Suspense|   2|
|             Horror|   2|
|          Adventure|   3|
|             Action|   3|
|            Western|   3|
|        Documentary|   3|
|    Romantic Comedy|   4|
|Concert/Performance|   4|
|              Drama|   5|
+-------------------+----+



In [26]:
movies2.write.option("header",True).mode("overwrite").csv('./temp_space/movies')

## partitionBy
- 저장시 partitionBy 컬럼에 따라 폴더가 나뉘어져 들어가는데 end단에 저장된 파일에는 그 컬럼이 빠져있다. 

In [45]:
movies = spark.read.option("inferSchema", "true").json("./dataset/movies.json")

In [25]:
movies.write.option("header",True).partitionBy("Major_Genre").mode("overwrite").csv('./temp_space/movies2')

## DataFrame.transform() - DF에 대해 가공할 내용들을 함수로 정의하여 한방에 적용

In [27]:
movies = spark.read.option("inferSchema", "true").json("./dataset/movies.json")

movies.show(5)

+--------------------+--------+-----------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+-------------------+--------------------+------------+--------+---------------+
|       Creative_Type|Director|Distributor|IMDB_Rating|IMDB_Votes|MPAA_Rating|Major_Genre|Production_Budget|Release_Date|Rotten_Tomatoes_Rating|Running_Time_min|             Source|               Title|US_DVD_Sales|US_Gross|Worldwide_Gross|
+--------------------+--------+-----------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+-------------------+--------------------+------------+--------+---------------+
|                null|    null|   Gramercy|        6.1|      1071|          R|       null|          8000000|   12-Jun-98|                  null|            null|               null|      The Land Girls|        null|  146083|         146083|
|                null|    null|     

In [83]:
def df_groupby(df, cols: str):
    df_ = df.groupBy(cols).count().orderBy("count", ascending=False)
    print(df_.count())
    return df_

In [79]:
def df_concat(df, cols: str, letter: str):
    df_ = df.withColumn("add", F.concat(F.col(cols), F.lit(letter)))
    return df_.select(cols, "add")

In [84]:
movies.transform(df_groupby, "Creative_Type").show(3)

10
+--------------------+-----+
|       Creative_Type|count|
+--------------------+-----+
|Contemporary Fiction| 1453|
|                null|  446|
|  Historical Fiction|  350|
+--------------------+-----+
only showing top 3 rows



In [37]:
movies.transform(df_concat, "Major_Genre", "@@@").show(3)

+-----------+---------+
|Major_Genre|      add|
+-----------+---------+
|       null|     null|
|      Drama| Drama@@@|
|     Comedy|Comedy@@@|
+-----------+---------+
only showing top 3 rows



## functions.transform -> array 형식으로 들어가있는 row를 다룰 때 사용한다.

In [67]:
movies2 = movies.na.drop(how='all', subset = ['Major_Genre', 'Distributor'])

In [68]:
movies3 = movies2.groupBy("Distributor").agg(F.collect_set(F.col("Major_Genre")).alias("Major_Genre2"))
movies3.show(3, False)

+----------------+------------------------------------------------------------------------------------------------------------------+
|Distributor     |Major_Genre2                                                                                                      |
+----------------+------------------------------------------------------------------------------------------------------------------+
|null            |[Action, Adventure, Romantic Comedy, Comedy, Drama, Horror, Documentary, Black Comedy, Musical, Thriller/Suspense]|
|20th Century Fox|[Action, Adventure, Romantic Comedy, Comedy, Drama, Horror, Western, Musical, Thriller/Suspense]                  |
|3D Entertainment|[Documentary]                                                                                                     |
+----------------+------------------------------------------------------------------------------------------------------------------+
only showing top 3 rows



In [72]:
movies3.select(F.transform("Major_Genre2", lambda x: F.upper(x)).alias('trans')).show(5, False)

+------------------------------------------------------------------------------------------------------------------+
|trans                                                                                                             |
+------------------------------------------------------------------------------------------------------------------+
|[ACTION, ADVENTURE, ROMANTIC COMEDY, COMEDY, DRAMA, HORROR, DOCUMENTARY, BLACK COMEDY, MUSICAL, THRILLER/SUSPENSE]|
|[ACTION, ADVENTURE, ROMANTIC COMEDY, COMEDY, DRAMA, HORROR, WESTERN, MUSICAL, THRILLER/SUSPENSE]                  |
|[DOCUMENTARY]                                                                                                     |
|[ACTION]                                                                                                          |
|[DRAMA]                                                                                                           |
+---------------------------------------------------------------

## broadcast
- A와 B를 Join 할때, A의 크기가 B에 비해 너무나 큰 상황일때는 단순 join으로 둘을 엮는게 비효율적일 수 있다. 특히 A의 어떠한 value를 B의 특정 value로 치환하고자 하는 목적일때는 더더욱 그렇다.
- 이럴 때는 B를 broadcast 변수로 만들어 모든 node에 복제한 후 Join하는 것과 동일한 효과를 shuffle 없이 얻을 수 있다.
- 단순히 join을 하게되면 shuffle이 일어나 broadcast를 쓴 것에 비해 효율이 낮다.

#### 1) join 상황과 비교

In [43]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark =  SparkSession \
        .builder \
        .appName('pyspark-broadcast') \
        .enableHiveSupport() \
        .getOrCreate()

In [39]:
data = [("101", 'Department 1'),
        ("102", 'Department 2'),
        ("103", 'Department 3'),
        ("104", 'Department 4'),
        ("105", 'Department 5'),
        ("106", 'Department 6'),
        ("107", 'Department 7')]

columns = ['department_id', 'dept_name']

df = spark.createDataFrame(data = data, schema = columns)

# df.show()

In [40]:
emp = spark.read.csv("/home/workspace/dataset/emp.csv", header = True)

# emp.show(2)

In [41]:
result1 = emp.join(df, ["department_id"], 'left')

result1.show()

+-------------+-----------+-------------+---+------+------+----------+------------+
|department_id|employee_id|         name|age|gender|salary| hire_date|   dept_name|
+-------------+-----------+-------------+---+------+------+----------+------------+
|          101|        001|     John Doe| 30|  Male| 50000|2015-01-01|Department 1|
|          101|        002|   Jane Smith| 25|Female| 45000|2016-02-15|Department 1|
|          101|        007|James Johnson| 42|  Male| 70000|2012-03-15|Department 1|
|          107|        014|    Emily Lee| 26|Female| 46000|2019-01-01|Department 7|
|          107|        016|  Kelly Zhang| 30|Female| 49000|2018-04-01|Department 7|
|          104|        010|     Lisa Lee| 27|Female| 47000|2018-08-01|Department 4|
|          104|        011|   David Park| 38|  Male| 65000|2015-11-01|Department 4|
|          104|        018|    Nancy Liu| 29|Female| 50000|2017-06-01|Department 4|
|          102|        003|    Bob Brown| 35|  Male| 55000|2014-05-01|Depart

In [42]:
spark.stop()

#### 2) Broadcast 상황

In [44]:
dept_names = {"101" : 'Department 1', 
              "102" : 'Department 2', 
              "103" : 'Department 3', 
              "104" : 'Department 4',
              "105" : 'Department 5', 
              "106" : 'Department 6', 
              "107" : 'Department 7'}

In [45]:
broadcast_dept_names = spark.sparkContext.broadcast(dept_names)

In [46]:
print(type(broadcast_dept_names))
broadcast_dept_names.value

<class 'pyspark.broadcast.Broadcast'>


{'101': 'Department 1',
 '102': 'Department 2',
 '103': 'Department 3',
 '104': 'Department 4',
 '105': 'Department 5',
 '106': 'Department 6',
 '107': 'Department 7'}

In [47]:
@F.udf
def get_dept_names(dept_id):
    return broadcast_dept_names.value.get(dept_id)

In [48]:
emp = spark.read.csv("/home/workspace/dataset/emp.csv", header = True)


In [49]:
emp_final = emp.withColumn("dept_name", get_dept_names(F.col("department_id")))

emp_final.show()

+-----------+-------------+-------------+---+------+------+----------+------------+
|employee_id|department_id|         name|age|gender|salary| hire_date|   dept_name|
+-----------+-------------+-------------+---+------+------+----------+------------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|Department 1|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|Department 1|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|Department 2|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|Department 2|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|Department 3|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|Department 3|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|Department 1|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|Department 2|
|        009|          103|      Tom Tan| 33|  Male| 58000|2016-06-01|Depart

## Accumulator

In [83]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark =  SparkSession \
        .builder \
        .appName('accumulator') \
        .enableHiveSupport() \
        .getOrCreate()

In [84]:
cars = spark.read.option("inferSchema", "true").json("./dataset/cars.json")


In [85]:
cars = cars.filter(F.col("Origin").isNotNull())

In [86]:
cars.filter(F.col("Origin") == 'USA').agg(F.sum("Cylinders")).show()

+--------------+
|sum(Cylinders)|
+--------------+
|          1596|
+--------------+



In [87]:
accum = spark.sparkContext.accumulator(0)

def sum_horsepower(origin, cyl):
    if origin == "USA":
        accum.add(cyl)

In [88]:
# 기준 컬럼에 null이 있으면 에러가 난다.

cars.foreach(lambda row : sum_horsepower(row.Origin, row.Cylinders))

In [89]:
accum.value

1596

In [82]:
spark.stop()